In [3]:
import findspark
findspark.init()
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
from pyspark.sql.types import *
#from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('Customers').getOrCreate()
from pyspark.ml.regression import LinearRegression
dataset = spark.read.csv("C:\\Users\\AJAY\\Desktop\\Ecommerce_Customers.csv", inferSchema=True,header=True)

In [6]:
dataset

DataFrame[Email: string, Address: string, Avg Session Length: double, Time on App: double, Time on Website: double, Length of Membership: double, Yearly Amount Spent: double]

In [7]:
dataset.show()

+--------------------+--------------------+------------------+-----------+---------------+--------------------+-------------------+
|               Email|             Address|Avg Session Length|Time on App|Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+------------------+-----------+---------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|       34.49726773|12.65565115|    39.57766802|         4.082620633|         587.951054|
|   hduke@hotmail.com|4547 Archer Commo...|       31.92627203|11.10946073|    37.26895887|         2.664034182|        392.2049334|
|    pallen@yahoo.com|24645 Valerie Uni...|       33.00091476|11.33027806|    37.11059744|         4.104543202|        487.5475049|
|riverarebecca@gma...|1414 David Throug...|       34.30555663|13.71751367|    36.72128268|         3.120178783|         581.852344|
|mstephens@davidso...|14023 Rodriguez P...|       33.33067252|12.79518855|  

In [8]:
dataset.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [9]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler


In [12]:
featureassembler = VectorAssembler(inputCols=["Avg Session Length","Time on App","Time on Website","Length of Membership"],
                                   outputCol = "Independent Feature")

In [13]:
output=featureassembler.transform(dataset)

In [14]:
output

DataFrame[Email: string, Address: string, Avg Session Length: double, Time on App: double, Time on Website: double, Length of Membership: double, Yearly Amount Spent: double, Independent Feature: vector]

In [15]:
output.show()

+--------------------+--------------------+------------------+-----------+---------------+--------------------+-------------------+--------------------+
|               Email|             Address|Avg Session Length|Time on App|Time on Website|Length of Membership|Yearly Amount Spent| Independent Feature|
+--------------------+--------------------+------------------+-----------+---------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|       34.49726773|12.65565115|    39.57766802|         4.082620633|         587.951054|[34.49726773,12.6...|
|   hduke@hotmail.com|4547 Archer Commo...|       31.92627203|11.10946073|    37.26895887|         2.664034182|        392.2049334|[31.92627203,11.1...|
|    pallen@yahoo.com|24645 Valerie Uni...|       33.00091476|11.33027806|    37.11059744|         4.104543202|        487.5475049|[33.00091476,11.3...|
|riverarebecca@gma...|1414 David Throug...|       34.30555663|13.71751367|    36.7

In [16]:
output.select("Independent Feature").show()

+--------------------+
| Independent Feature|
+--------------------+
|[34.49726773,12.6...|
|[31.92627203,11.1...|
|[33.00091476,11.3...|
|[34.30555663,13.7...|
|[33.33067252,12.7...|
|[33.87103788,12.0...|
|[32.0215955,11.36...|
|[32.73914294,12.3...|
|[33.9877729,13.38...|
|[31.93654862,11.8...|
|[33.99257277,13.3...|
|[33.87936082,11.5...|
|[29.53242897,10.9...|
|[33.19033404,12.9...|
|[32.38797585,13.1...|
|[30.73772037,12.6...|
|[32.1253869,11.73...|
|[32.33889932,12.0...|
|[32.18781205,14.7...|
|[32.61785606,13.9...|
+--------------------+
only showing top 20 rows



In [17]:
finalized_data = output.select("Independent Feature","Yearly Amount Spent")

In [18]:
finalized_data.show()

+--------------------+-------------------+
| Independent Feature|Yearly Amount Spent|
+--------------------+-------------------+
|[34.49726773,12.6...|         587.951054|
|[31.92627203,11.1...|        392.2049334|
|[33.00091476,11.3...|        487.5475049|
|[34.30555663,13.7...|         581.852344|
|[33.33067252,12.7...|         599.406092|
|[33.87103788,12.0...|        637.1024479|
|[32.0215955,11.36...|        521.5721748|
|[32.73914294,12.3...|        549.9041461|
|[33.9877729,13.38...|         570.200409|
|[31.93654862,11.8...|        427.1993849|
|[33.99257277,13.3...|        492.6060127|
|[33.87936082,11.5...|        522.3374046|
|[29.53242897,10.9...|        408.6403511|
|[33.19033404,12.9...|        573.4158673|
|[32.38797585,13.1...|        470.4527333|
|[30.73772037,12.6...|        461.7807422|
|[32.1253869,11.73...|        457.8476959|
|[32.33889932,12.0...|        407.7045475|
|[32.18781205,14.7...|        452.3156755|
|[32.61785606,13.9...|        605.0610388|
+----------

In [20]:
train_data, test_data = finalized_data.randomSplit([.75,.25])
regressor = LinearRegression(featuresCol=  "Independent Feature", labelCol = "Yearly Amount Spent")
regressor = regressor.fit(train_data)

In [21]:
regressor.coefficients

DenseVector([25.938, 38.705, 0.4762, 61.452])

In [22]:
regressor.intercept

-1059.4504515391627

In [23]:
pred_results = regressor.evaluate(test_data)
pred_results.predictions.show()

+--------------------+-------------------+------------------+
| Independent Feature|Yearly Amount Spent|        prediction|
+--------------------+-------------------+------------------+
|[30.73772037,12.6...|        461.7807422| 450.5141271237194|
|[31.12809005,13.2...|        557.2526867| 563.9956602957027|
|[31.1695068,13.97...|        427.3565308| 416.9050657280484|
|[31.28344748,12.7...|        591.7810894|  568.927139195861|
|[31.3123496,11.68...|         463.591418|444.30539656082783|
|[31.38958548,10.9...|        410.0696111| 409.1036036049313|
|[31.44597248,12.8...|        484.8769649|481.62556295208196|
|[31.52575242,11.3...|        443.9656268|449.02589418826983|
|[31.5261979,12.04...|        409.0945262| 417.8227723919872|
|[31.53160448,13.3...|        436.5156057|432.64578338465526|
|[31.57413802,12.9...|        544.4092722| 558.1548369985828|
|[31.6005122,12.22...|        479.1728515|460.65510610106776|
|[31.65480968,13.0...|        475.2634237|468.31347091679163|
|[31.765